In [1]:
seed = 42

# Import libraries
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

import random
random.seed(seed)
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.metrics import accuracy_score, f1_score, roc_curve, roc_auc_score, precision_recall_curve, auc, confusion_matrix
from imblearn.over_sampling import SMOTE

from sklearn.feature_selection import mutual_info_classif, SelectKBest, SelectPercentile, f_classif, f_regression, SelectFromModel
from scipy.spatial.distance import pdist, squareform
from scipy.stats import ttest_ind
from xgboost import XGBClassifier
import statistics
from sklearn.decomposition import PCA
from sklearn.metrics import balanced_accuracy_score

import pickle

In [2]:

file_path = "../CSV/data_rad_clin_DEF.csv"

data = pd.read_csv(file_path)
labels_column = data['label']
labels = labels_column.astype(int).tolist()
labels=np.array(labels)

# Estrazione dei numeri dai nomi dei pazienti
loaded_patients = data['IDs_new'].str.extract(r'(\d+)').astype(int).squeeze().tolist()

print("Labels:", labels)
print("Number of labels:", len(labels))
print("Patient Names: ", loaded_patients )


FileNotFoundError: [Errno 2] No such file or directory: '../CSV/data_rad_clin_DEF.csv'

In [ ]:
import zipfile
import pandas as pd
import numpy as np

# Percorso del file .npy
file_path = "../indici_aree/aree_05.npy"
arraytuple = np.load(file_path)

# Converti il primo elemento di ogni tupla in int
arraytuple = [(int(t[0]), t[1], t[2]) for t in arraytuple]

# Percorso del file zip
zip_file_path = "../CSV/EncodersAllSlices/RESNET50_ALL_SLICES.zip"

# Apri il file zip e leggi il CSV direttamente dalla memoria
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Ottieni il nome del file CSV dentro lo zip
    csv_filename = zip_ref.namelist()[0]
    # Leggi il file CSV direttamente dalla memoria
    with zip_ref.open(csv_filename) as csv_file:
        data = pd.read_csv(csv_file)

# Droppa l'ultima colonna
data = data.drop(data.columns[-1], axis=1)

# Filtra il DataFrame per mantenere solo le righe che corrispondono agli elementi di arraytuple
filtered_data = data[
    data.apply(lambda row: any((row['Patient'] == (t[0]) and row['Slice'] == t[1]) for t in arraytuple), axis=1)
]

# Filter columns that start with 'original' or 'dentro'
filtered_patients = []

for patient_id in loaded_patients:
    # Filter the data for the specific patient
    filtered_patient_data = filtered_data[filtered_data['Patient'] == patient_id]
    
    # Dopo aver filtrato per paziente, rimuovi le prime due colonne ('Patient' e 'Slice')
    filtered_patient_data = filtered_patient_data.drop(filtered_patient_data.columns[:2], axis=1)
    
    slices = []
    
    for _, slice_row in filtered_patient_data.iterrows():
        # Select only the filtered columns for each slice
        slice_features = slice_row.tolist()
        slices.append(slice_features)
    
    filtered_patients.append(slices)

# Stampa per verificare che le prime due colonne siano state rimosse
print(filtered_patient_data.head())


In [ ]:
## funzione per creare array da array di array
def continue_array(filtered_patients, labels):
    all_features = []
    for patient in filtered_patients:
        for image_features in patient:
            all_features.append(image_features)

    all_features_array = np.array(all_features)
    expanded_labels = []
    expanded_patient_ids = []

    for i in range(len(filtered_patients)):
        num_images = len(filtered_patients[i])
        expanded_labels.extend([labels[i]] * num_images)
        expanded_patient_ids.extend([loaded_patients[i]] * num_images)

    expanded_labels_array = np.array(expanded_labels)
    expanded_patient_ids_array = np.array(expanded_patient_ids)

    return all_features_array, expanded_labels_array, expanded_patient_ids_array


## funzioni per feature correlation
def filter_highly_correlated_features(df, corr, threshold=0.85):
    columns = np.full((corr.shape[0],), True, dtype=bool)
    removed_features = []

    for i in range(corr.shape[0]):
        for j in range(i + 1, corr.shape[0]):
            if corr.iloc[i, j] >= threshold:
                if columns[j]:
                    columns[j] = False
                    removed_features.append(df.columns[j])

    return removed_features


def perform_correlation(z_train, y_train, numero = 32, threshold = 0.85):
    all_images, _, _= continue_array(z_train, y_train)

    df = pd.DataFrame(all_images, columns=[f'feature_{i}' for i in range(numero)])

    corr_matrix = df.corr()

    features_selected = filter_highly_correlated_features(df, corr_matrix, threshold)
    
    return features_selected

## funzione per rimuovere le features con p_value maggiore della treshold
def select_features_by_p_value(x_train_expanded, y_train_expanded, p_value_threshold=0.05):

    p_values = []
    num_features = x_train_expanded.shape[1]

    for i in range(num_features):
        feature = x_train_expanded[:, i]
        group_0 = feature[y_train_expanded == 0]
        group_1 = feature[y_train_expanded == 1]
        t_stat, p_val = ttest_ind(group_0, group_1, equal_var=False)
        p_values.append(p_val)

    p_values = np.array(p_values)

    selected_features_indices = np.where(p_values < p_value_threshold)[0]

    sorted_indices = selected_features_indices[np.argsort(p_values[selected_features_indices])]

    x_train_expanded = x_train_expanded[:, sorted_indices]

    return x_train_expanded, sorted_indices



## funzione per rimozione di features specifiche
def remove_features_from_patients(patients, features_to_remove):
    feature_indices_to_remove = [int(feature.split('_')[1]) for feature in features_to_remove]
    
    final_patients = []
    for patient in patients:
        new_patients = []
        for image_features in patient:
            new_patient = np.delete(image_features, feature_indices_to_remove, axis=0)
            new_patients.append(new_patient)
        final_patients.append(np.array(new_patients))    

    return final_patients


## FEATURE SELECTION LASSO
def select_features_with_lasso(X, y, alpha=0.001):
    
    lasso = Lasso(alpha=alpha)
    lasso.fit(X, y)
    coefficients = lasso.coef_
    selected_features = np.where(coefficients != 0)[0]
    X_selected = X[:, selected_features]

    return X_selected, selected_features

## FEATURE SELECTION LOGISTIC
def logistic_regression_feature_selection(X, y, num_features):
    lr = LogisticRegression(max_iter=2000, random_state=42)
    lr.fit(X, y)
    coef_abs = np.abs(lr.coef_)
    feature_importances = np.mean(coef_abs, axis=0)
    selected_features = feature_importances.argsort()[-num_features:][::-1]
    X_selected = X[:, selected_features]
    return X_selected, selected_features

## FEATURE SELECTION MRMR
def mrmr_feature_selection(X, y, num_features):
    mi = mutual_info_classif(X, y, random_state=42)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    distances = squareform(pdist(X_scaled.T, 'euclidean'))
    
    selected_features = []
    selected_indices = []

    first_feature_index = np.argmax(mi)
    selected_features.append(first_feature_index)
    selected_indices.append(first_feature_index)
    
    for _ in range(num_features - 1):
        max_relevance = -np.inf
        selected_feature_index = -1
        
        for i in range(X.shape[1]):
            if i in selected_indices:
                continue
            
            relevance = mi[i]
            redundancy = np.mean(distances[i, selected_indices])
            
            mrmr_score = relevance - redundancy
            
            if mrmr_score > max_relevance:
                max_relevance = mrmr_score
                selected_feature_index = i
        
        selected_features.append(selected_feature_index)
        selected_indices.append(selected_feature_index)

    X_selected = X[:, selected_indices]
    return X_selected, selected_indices

## FEATURE SELECTION RANDOM FOREST
def rf_feature_selection(X, y, num_features):
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X, y)
    feature_importances = rf.feature_importances_
    selected_features = np.argsort(feature_importances)[-num_features:][::-1]
    X_selected = X[:, selected_features]
    return X_selected, selected_features


## FEATURE SELECTION P_VALUE
# Seleziona e ordina le feature basate sui p-value con un test t di Student poi 
# ordina le feature in base al p-value in ordine crescente e seleziona le prime `num_features` caratteristiche.

def p_value_feature_selection(x_train_expanded, y_train_expanded, num_features):
    p_values = []
    num_features_total = x_train_expanded.shape[1]

    # Calcolo dei p-value per ciascuna feature
    for i in range(num_features_total):
        feature = x_train_expanded[:, i]
        group_0 = feature[y_train_expanded == 0]
        group_1 = feature[y_train_expanded == 1]
        t_stat, p_val = ttest_ind(group_0, group_1, equal_var=False)
        p_values.append(p_val)


    p_values = np.array(p_values)

    # Ordinare tutte le caratteristiche in base ai p-value (dal più piccolo al più grande)
    sorted_indices = np.argsort(p_values)
    sorted_indices = sorted_indices[:num_features]

    x_train_selected = x_train_expanded[:, sorted_indices]

    return x_train_selected, sorted_indices



## funzione per lasciare solo le features indicate per array di array
def keep_features_in_patients(patients, features_to_keep):

    feature_indices_to_keep = [int(feature) for feature in features_to_keep]

    final_patients = []
    for patient in patients:
        new_patients = []
        for image_features in patient:
            new_patient = np.take(image_features, feature_indices_to_keep, axis=0)
            new_patients.append(new_patient)
        final_patients.append(np.array(new_patients))

    return final_patients


## funzione per lasciare solo le features indicate per array
def filter_patients_features(filtered_patients, selected_features):
    filtered_patients_selected = []
    
    for patient_features in filtered_patients:
        # Select only the features specified in selected_features
        patient_features_selected = patient_features[:, selected_features]
        filtered_patients_selected.append(patient_features_selected)

    return filtered_patients_selected


def classifierinitialization(classifier):
    if classifier == 'RandomForest':
                            classi = RandomForestClassifier(n_estimators=100, random_state=42)
    elif classifier == 'Logistic':
                            classi = LogisticRegression(random_state=42, max_iter=2000)
    elif classifier == 'SVM':
                            classi = SVC(kernel='rbf', probability=True, random_state=42)
    elif classifier == 'XgBoost':
                            classi = XGBClassifier(random_state=42)
    elif classifier == 'MLP':
                            classi = MLPClassifier(hidden_layer_sizes=(128, 64, 32), max_iter=1000, random_state=42, early_stopping=True, learning_rate='adaptive', activation = 'logistic')
    elif classifier == 'ensemble':
                            rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
                            logistic_model = LogisticRegression(random_state=42, max_iter=2000)
                            svc_model = SVC(kernel='rbf', probability=True, random_state=42)
                            classi = VotingClassifier(
                                estimators=[
                                    ('random_forest', rf_model),
                                    ('logistic', logistic_model),
                                    ('svc', svc_model)
                                ],
                                voting='soft'
                                )
    return classi

## funzione per effettuarr majority voting o mean su tutte le slice di un paziente, per passare da una predizione sulla slice
## alla predizione per il paziente

def prob_to_binary(predictions_proba, patient_scores, threshold, mode):
    final_predictionarray = []          
    
    if mode == 'MV':  # Majority Voting
        for p in predictions_proba:
            test_patient_predictions = []
            for proba in p:               
                predictions_binary = 1 if proba[0][1] > threshold else 0
                test_patient_predictions.append(predictions_binary)
            count_0 = np.sum(np.array(test_patient_predictions) == 0) 
            count_1 = np.sum(np.array(test_patient_predictions) == 1)                                   
            final_prediction = 0 if count_0 > count_1 else 1
            final_predictionarray.append(final_prediction)
    
    elif mode == 'Mean':  # Mean of probabilities
        for score in patient_scores:
            predictions_binary = 1 if score > threshold else 0
            final_predictionarray.append(predictions_binary)
    
    elif mode == 'Max':  # Maximum probability across both classes
        for p in predictions_proba:
            max_proba = None
            max_slice = None

            # Itera su ogni slice per trovare la massima probabilità (indipendentemente dalla classe)
            for proba in p:
                class_0_prob = proba[0][0]  # Probabilità della classe 0
                class_1_prob = proba[0][1]  # Probabilità della classe 1
                
                # Trova la probabilità massima tra entrambe le classi per ciascuna slice
                slice_max_proba = max(class_0_prob, class_1_prob)

                # Trova la slice con la probabilità massima
                if max_proba is None or slice_max_proba > max_proba:
                    max_proba = slice_max_proba
                    max_slice = proba  # Memorizza la slice con la probabilità massima

            # Ora usa la probabilità della classe 1 della slice con la probabilità massima per il confronto con la soglia
            predictions_binary = 1 if max_slice[0][1] > threshold else 0
            final_predictionarray.append(predictions_binary)




    return final_predictionarray



def classification_method(selector, alpha, classifier, x_train_expanded, y_train_expanded, patients_test, y_test, features_test, num_features, modePrediction, thresholds=np.arange(0.001, 0.501, 0.001), mode = "Val", selected_features= [0]):

    #print(features_test)
    if(mode == "Val"):
            selected_features = None 
            
            if num_features != len(x_train_expanded[0]) or alpha != 0:
                if selector == "lasso":
                    X_selected, selected_features = select_features_with_lasso(x_train_expanded, y_train_expanded, alpha)
                    if(len(selected_features)==0):
                            return 0
                elif selector == "logistic": 
                    X_selected, selected_features = logistic_regression_feature_selection(x_train_expanded, y_train_expanded, num_features)
                elif selector == "mrmr":
                    X_selected, selected_features = mrmr_feature_selection(x_train_expanded, y_train_expanded, num_features)
                elif selector == "rf":
                    X_selected, selected_features = rf_feature_selection(x_train_expanded, y_train_expanded, num_features)
                elif selector=="p_value":
                    X_selected, selected_features= p_value_feature_selection(x_train_expanded,y_train_expanded, num_features)
                else:
                    print("Wrong selector. Choose between: mrmr, rf, logistic, p_value, lasso")
                    return
                
                #features_test = filter_patients_features(features_test, selected_features)
                features_test = keep_features_in_patients(features_test, selected_features)
                keep_features_in_patients
                #print(f"features_test {features_test}")
            else:
                X_selected = x_train_expanded
                selected_features = list(range(len(x_train_expanded[0])))  # Selezioniamo tutte le feature se non si fa feature selection
            number_features = len(selected_features) 
        
            smote = SMOTE(random_state=42)

            X_resampled, y_resampled = smote.fit_resample(X_selected, y_train_expanded)
            classifier.fit(X_resampled, y_resampled)

    patient_scores=[]
    temp_array=[]

    for x in range(len(patients_test)):
        patient_predictions = []
        patient_predictions1 = []
        r=len(features_test[x])
        for i in range(r):
            dato = features_test[x][i].reshape(1, -1)
            prediction = classifier.predict_proba(dato)
            prediction1 = classifier.predict_proba(dato)[:,1]
            patient_predictions.append(prediction)
            patient_predictions1.append(prediction1)
            mean=np.mean(patient_predictions1)

        patient_scores.append(mean) ## contiene la media delle probabilità
        temp_array.append(patient_predictions) ## contiene tutte le probabilità
    
    

    best_f1_score = 0
    best_threshold = None
    #best_precision=0
    #best_recall=0
    best_prediction=[]
    
    # Valuta le performance utilizzando diverse threshold
    if isinstance(thresholds, (int, float, complex)):
        thresholds=[thresholds]

    ## se non viene specificato usi il parametro di default 
    ## viene calcolata la threshold che porta a un f1 migliore e effettuata la prediction con quella
    if(len(thresholds)!=1):
        for threshold in thresholds:
            binary_predictions = prob_to_binary(temp_array, patient_scores, threshold, modePrediction)
            f1 = f1_score(y_test, binary_predictions)
            if f1 > best_f1_score:
                best_f1_score = f1
                best_threshold = threshold
                #best_precision = precision_score(y_test, binary_predictions)
                #best_recall = recall_score(y_test, binary_predictions)
                best_prediction=binary_predictions

    else: ## qui vuol dire che è il set di test che usa la threshold migliore che viene passata dal validation
        best_threshold = thresholds
        best_prediction=prob_to_binary(temp_array, patient_scores, best_threshold, modePrediction)
        best_f1_score = f1_score(y_test, best_prediction)
        #best_precision = precision_score(y_test, best_prediction)
        #best_recall = recall_score(y_test, best_prediction)

    #print(f"La migliore threshold è {best_threshold} con f1score di {best_f1_score} e precision {best_precision} e recall {best_recall}.")
    y_test= np.array(y_test)
    best_prediction=np.array(best_prediction)
    test_accuracy = accuracy_score(y_test, best_prediction)

    #precision, recall, _ = precision_recall_curve(y_test, patient_scores) ## utili per ricostruire grafici
    pr_auc = average_precision_score(y_test, patient_scores)

    #fpr, tpr, _ = roc_curve(y_test, best_prediction)
    roc_auc= roc_auc_score(y_test, patient_scores)

    # Calcola la balanced accuracy
    bal_acc = balanced_accuracy_score(y_test, best_prediction)

    conf= confusion_matrix(y_test, best_prediction)


    best_case = {
                    'alpha': alpha,
                    'num_features': number_features,
                    'selected_features': selected_features,
                    'pr_auc': pr_auc,
                    'roc_auc': roc_auc,
                    'f1': best_f1_score,
                    'accuracy': test_accuracy,
                    'confusion_matrix': conf,
                    'best_threshold': best_threshold,
                    'balanced accuracy': bal_acc
                }
    
    return best_case


In [ ]:

# patients_train contiene il nome dei pazienti (5,12 etc)
# y_train contiene le labels
# features_train contiene array di array dove ogni paziente ha varie immagini rappresentate da n features

#patients_train1, patients_test, y_train1, y_test, features_train1, features_test= train_test_split(loaded_patients, labels, patients, test_size=0.3, shuffle=False)

patients_train1, patients_test, y_train1, y_test, features_train1, features_test= train_test_split(loaded_patients, labels, filtered_patients, test_size=0.3, shuffle=False)

print("Number of train patients: ", len(features_train1))
print("Number of test patients: ", len(features_test))


print("Number of features for every image: ", len(features_train1[0][0]) )


In [ ]:
### correlation 0.8
starting_features = len(features_train1[0][0])
features=perform_correlation(features_train1, y_train1, starting_features, 0.9)

final_patients_train1=remove_features_from_patients(features_train1, features)
final_patients_test=remove_features_from_patients(features_test, features)

print(f"correlation ha trovato {final_patients_train1[0].shape[1]} features\n")

### p-value 0.01
x_train_expanded1, y_train_expanded1, _ = continue_array(final_patients_train1, y_train1)
x_train_expanded1, sf= select_features_by_p_value(x_train_expanded1, y_train_expanded1, 0.05)
print(f"Scelte {len(sf)} features dal p_value\n")
print("x_train_expanded1", x_train_expanded1.shape)

final_patients_test=keep_features_in_patients(final_patients_test, sf)
final_patients_train1=keep_features_in_patients(final_patients_train1, sf)

print("final_patients_test", final_patients_test[0].shape)
print("final_patients_train1", final_patients_train1[0].shape)

patients_train1 = np.array(patients_train1)


In [ ]:
import zipfile
import pandas as pd
import numpy as np
import statistics
import functools

def select_and_save_features(input_npy, input_zip, loaded_patients, results_val_others, results_val_lasso, original_features, output_csv_filename):
    # Carica il file .npy contenente le tuple (Paziente, Slice, altro)
    arraytuple = np.load(input_npy)
    # Converti il primo elemento di ogni tupla in intero
    arraytuple = [(int(t[0]), t[1], t[2]) for t in arraytuple]

    # Apri il file zip e leggi il CSV
    with zipfile.ZipFile(input_zip, 'r') as zip_ref:
        # Ottieni il nome del file CSV dentro lo zip
        csv_filename = zip_ref.namelist()[0]
        # Leggi il file CSV direttamente dalla memoria
        with zip_ref.open(csv_filename) as csv_file:
            data = pd.read_csv(csv_file)

    # Droppa l'ultima colonna (se necessario)
    data = data.drop(data.columns[-1], axis=1)

    # Filtra il DataFrame per mantenere solo le righe che corrispondono agli elementi di arraytuple
    filtered_data = data[
        data.apply(lambda row: any((row['Patient'] == t[0] and row['Slice'] == t[1]) for t in arraytuple), axis=1)
    ]

    # Inizia la grid search con i selettori non-lasso
    num_features_range = list(range(1, 30))
    grid_results_others = {}
    grid_results_lasso = {}

    selectors_others = ['mrmr', 'rf', 'logistic']
    classifiers = set([res['classifier'] for res in results_val_others] + [res['classifier'] for res in results_val_lasso])
    alpha_values = list(set(res['alpha'] for res in results_val_lasso if 'alpha' in res))
    modes = set([res['mode'] for res in results_val_others + results_val_lasso])  # Assumi che 'mode' sia un parametro comune

    # Itera su results_val_others
    for classifier in classifiers:
        for selector in selectors_others:
            for mode in modes:
                for num_features in num_features_range:
                    filtered_results = [
                        res for res in results_val_others if (
                            res['classifier'] == classifier and
                            res['selector'] == selector and
                            res['num_features'] == num_features and
                            res['mode'] == mode
                        )
                    ]
                    if filtered_results:
                        f1_values = [res['f1'] for res in filtered_results]
                        balaccuracy_values = [res['balanced accuracy'] for res in filtered_results]
                        roc_values = [res['roc_auc'] for res in filtered_results]

                        avg_f1 = sum(f1_values) / len(f1_values)
                        avg_balaccuracy = sum(balaccuracy_values) / len(balaccuracy_values)
                        avg_roc = sum(roc_values) / len(roc_values)

                        std_f1 = statistics.stdev(f1_values) if len(f1_values) > 1 else 0
                        std_balaccuracy = statistics.stdev(balaccuracy_values) if len(balaccuracy_values) > 1 else 0
                        std_roc_auc = statistics.stdev(roc_values) if len(roc_values) > 1 else 0

                        grid_results_others[(classifier, selector, num_features, mode)] = {
                            'avg_f1': avg_f1,
                            'std_f1': std_f1,
                            'avg_balaccuracy': avg_balaccuracy,
                            'std_balaccuracy': std_balaccuracy,
                            'avg_roc_auc': avg_roc,
                            'std_roc_auc': std_roc_auc
                        }

    # Itera su results_val_lasso
    for classifier in classifiers:
        for mode in modes:
            for alpha in alpha_values:
                filtered_results = [
                    res for res in results_val_lasso if (
                        res['classifier'] == classifier and
                        res['alpha'] == alpha and
                        res['mode'] == mode
                    )
                ]
                if filtered_results:
                    f1_values = [res['f1'] for res in filtered_results]
                    balaccuracy_values = [res['balanced accuracy'] for res in filtered_results]
                    roc_values = [res['roc_auc'] for res in filtered_results]

                    avg_f1 = sum(f1_values) / len(f1_values)
                    avg_balaccuracy = sum(balaccuracy_values) / len(balaccuracy_values)
                    avg_roc = sum(roc_values) / len(roc_values)

                    std_f1 = statistics.stdev(f1_values) if len(f1_values) > 1 else 0
                    std_balaccuracy = statistics.stdev(balaccuracy_values) if len(balaccuracy_values) > 1 else 0
                    std_roc_auc = statistics.stdev(roc_values) if len(roc_values) > 1 else 0

                    grid_results_lasso[(classifier, 'lasso', alpha, mode)] = {
                        'avg_f1': avg_f1,
                        'std_f1': std_f1,
                        'avg_balaccuracy': avg_balaccuracy,
                        'std_balaccuracy': std_balaccuracy,
                        'avg_roc_auc': avg_roc,
                        'std_roc_auc': std_roc_auc
                    }

    # Ordina i risultati
    sorted_results_others = sorted(grid_results_others.items(), key=lambda x: (x[1]['avg_balaccuracy'], x[1]['avg_roc_auc']), reverse=True)
    sorted_results_lasso = sorted(grid_results_lasso.items(), key=lambda x: (x[1]['avg_balaccuracy'], x[1]['avg_roc_auc']), reverse=True)

    sorted_results = sorted_results_others + sorted_results_lasso

    def sort_key(x):
        result1 = round(x[1]['avg_balaccuracy'], 2)
        result2 = round(x[1]['avg_roc_auc'], 2)
        return (result1, result2)

    sorted_results = sorted(sorted_results, key=sort_key, reverse=True)

    # Seleziona il miglior risultato
    best_combination = sorted_results[0]
    params, metrics = best_combination

    # Trova le selected_features per il fold 0
    selected_features = []
    if params[1] == 'lasso':
        for res in results_val_lasso:
            if (res['classifier'] == params[0] and res['alpha'] == params[2] and res['mode'] == params[3]):
                selected_features = res['selected_features']
                break
    else:
        for res in results_val_others:
            if (res['classifier'] == params[0] and res['selector'] == params[1] and res['num_features'] == params[2] and res['mode'] == params[3]):
                selected_features = res['selected_features']
                break

    # Seleziona gli indici originali delle feature
    selected_original_features = [original_features[i] for i in selected_features]

    # Seleziona le colonne corrispondenti dal dataframe filtrato
    df_features_selected = filtered_data.iloc[:, selected_original_features]

    # Aggiungi la colonna paziente e slice
    df_selected = pd.concat([filtered_data[['Patient', 'Slice']], df_features_selected], axis=1)

    # Rinomina 'Unnamed: 0' in 'Paziente'
    df_selected.rename(columns={'Patient': 'Paziente'}, inplace=True)

    # Ordina per il numero del paziente
    df_selected = df_selected.sort_values(by=['Paziente', 'Slice'])

    # Salva il file CSV
    df_selected.to_csv(output_csv_filename, index=False)

    print(f"Le feature selezionate sono state salvate nel file '{output_csv_filename}'")
